# Setup for the code generation

In [10]:
from dataclasses import dataclass
import scipy as sp
import itertools
from pathlib import Path

base_output_path = Path("./output")


def delta_recursion(alphabet: list, cur: set, n_delta_left: int, all_combinations: set):
    """Takes all combinations of two symbols from alphabet and adds them to a copy of the current combination.
    Then for every found combinations, removes the two symbols in the combination from the alphabet and recurses.
    If no kronecker deltas are left, adds the current combination to `all_combinations`.

    Args:
        alphabet (list): The list of available symbols
        cur (set): The current combination.
        n_delta (int): the number of delta functions that are left
        all_combinations (set): The set of index pairs under the delta function.
    """

    if n_delta_left >= 1:
        combinations = itertools.combinations(alphabet, 2)
        for p in combinations:
            cur_t = cur.copy()
            alphabet_t = alphabet.copy()
            alphabet_t.remove(p[0])
            alphabet_t.remove(p[1])
            cur_t.add(p)
            delta_recursion(
                alphabet_t.copy(),
                cur_t.copy(),
                n_delta_left=n_delta_left - 1,
                all_combinations=all_combinations,
            )
    else:
        for a in alphabet:
            cur.add(a)
        all_combinations.add(frozenset(cur))
        return None


def find_permutations(n_delta: int, n_r: int) -> set[frozenset[int]]:
    """
    Assume a tensor in Einstein notation with `n_delta` kronecker deltas and `n_r` position vectors.
    I.e `delta_{alpha beta} delta_{gamma delta} r_epsilon` would correspond to `n_delta = 2` and `n_r = 1`.
    This function finds all permuations of the indices that would lead to a new tensor.

    The type of the return value is a set of a frozenset since neither the order of `delta` functions and `r` (delta_{ab}r_c is equal to r_Cdelta_{ab}),
    nor the order of indices in the delta function matters (delta_{ab} is equal to delta_{ba})

    Args:
        n_delta (int): number of kronecker deltas
        n_r (int): number of position vectors

    Returns:
        permutations (set[frozenset[int]]): the permutations
    """

    n_symbols = 2 * n_delta + n_r
    alphabet = list(range(n_symbols))
    all_combinations = set()
    delta_recursion(alphabet, set(), n_delta, all_combinations)
    return all_combinations


@dataclass
class EinsteinTerm:
    """Represents a tensor in einstein summations of the form
        'prefactor/R^order * ( \delta_{ab} \delta_{cd} ... r_k r_l ... r_z + <permutations> )'
    with `n_delta` delta functions and `n_r` position vectors.
    The `permutations` refer to permuations of the ordered list of indices [abcde ...]
    """

    def __init__(self):
        self.permutations = set()
        self.prefactor = 0
        self.order = 0
        self.n_delta = 0
        self.n_r = 0


def T_Tensor(n: int) -> list[EinsteinTerm]:
    """
    Represents the `n`-th derivative of the 'T-Tensor', where T= 1/R = 1/sqrt(rx^2 + ry^2 + rz^2).
    For example at n=2, we have
        T_{ab} = \partia_{r_a} \partial_{r_b} 1/R.
    etc.

    Args:
        n (int): The order of the derivative

    Returns:
        list[EinsteinTerm]: result as a list of terms in einstein summation notation
    """

    einstein_terms = []

    if n % 2 == 0:
        lowest_order = n + 1
    else:
        lowest_order = n + 2

    highest_order = 2 * n + 1

    print(f"{lowest_order = }")
    print(f"{highest_order = }")

    # The overall 1/R^{exponent} scaling of the tensor
    r_scaling_exponent = n + 1
    print(f"{r_scaling_exponent = }")

    for l in range(int((n + 1) / 2), n + 1):
        order = 2 * l + 1
        print(order)

        # constant prefactor
        pref = (-1) ** l * sp.special.factorial2(2 * l - 1, exact=True)
        print(f"{pref = }")

        n_r = order - r_scaling_exponent
        n_delta_functions = int((n - n_r) / 2)

        print(f"{n_r = }")
        print(f"{n_delta_functions = }")

        permutations = find_permutations(n_delta_functions, n_r)
        print(permutations)

        term = EinsteinTerm()
        term.n_delta = n_delta_functions
        term.n_r = n_r
        term.prefactor = pref
        term.order = order
        term.permutations = permutations

        print(term)

        einstein_terms.append(term)

    return einstein_terms


def insert_separator(items: list[str], sep=",") -> str:
    """Returns a string with `sep` between each item of the list
    e.g insert_separator(  ["ab", "c", "d", "efg"], sep=", " ) -> "ab, c, d, efg"
    """

    if len(items) == 0:
        return ""

    if len(items) == 1:
        return items[0]

    if len(items) == 2:
        return f"{items[0]}{sep}{items[1]}"

    res = ""
    res += items[0]
    res += sep

    for i in items[1:-1]:
        res += i
        res += sep

    res += items[-1]
    return res

<>:68: SyntaxWarning: invalid escape sequence '\d'
<>:83: SyntaxWarning: invalid escape sequence '\p'
<>:68: SyntaxWarning: invalid escape sequence '\d'
<>:83: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_186037/368039975.py:68: SyntaxWarning: invalid escape sequence '\d'
  """Represents a tensor in einstein summations of the form
/tmp/ipykernel_186037/368039975.py:83: SyntaxWarning: invalid escape sequence '\p'
  """


# Latex expressions

In [11]:
def to_latex(einstein_terms: list[EinsteinTerm]):
    """Turns a list of EinsteinTerm into a latex string"""
    alphabet = [chr(i) for i in range(97, 97 + 24)]

    result_string = ""

    for t in einstein_terms:
        result_string += (
            "+ \\frac{"
            + f"{t.prefactor}"
            + "}{"
            + "R^{"
            + f"{t.order}"
            + "}}"
            + "\\left( "
        )

        counter = 0

        for permutation in t.permutations:
            if counter > 0:
                result_string += " + "
            counter += 1

            for indices in permutation:
                try:
                    s_1 = alphabet[indices[0]]
                    s_2 = alphabet[indices[1]]
                    result_string += "\\delta_{" + f"{s_1}{s_2}" + "}"
                except:
                    s = alphabet[indices]
                    result_string += "r_{" + f"{s}" + "}"

        result_string += "\\right)\n"

    return result_string


output_path = base_output_path / "latex"
output_path.mkdir(exist_ok=True, parents=True)
for n in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    with open(output_path / f"t{n}.txt", "w") as f:
        f.write(to_latex(T_Tensor(n)))

lowest_order = 1
highest_order = 1
r_scaling_exponent = 1
1
pref = 0
n_r = 0
n_delta_functions = 0
{frozenset()}
EinsteinTerm()
lowest_order = 3
highest_order = 3
r_scaling_exponent = 2
3
pref = -1
n_r = 1
n_delta_functions = 0
{frozenset({0})}
EinsteinTerm()
lowest_order = 3
highest_order = 5
r_scaling_exponent = 3
3
pref = -1
n_r = 0
n_delta_functions = 1
{frozenset({(0, 1)})}
EinsteinTerm()
5
pref = 3
n_r = 2
n_delta_functions = 0
{frozenset({0, 1})}
EinsteinTerm()
lowest_order = 5
highest_order = 7
r_scaling_exponent = 4
5
pref = 3
n_r = 1
n_delta_functions = 1
{frozenset({0, (1, 2)}), frozenset({(0, 1), 2}), frozenset({1, (0, 2)})}
EinsteinTerm()
7
pref = -15
n_r = 3
n_delta_functions = 0
{frozenset({0, 1, 2})}
EinsteinTerm()
lowest_order = 5
highest_order = 9
r_scaling_exponent = 5
5
pref = 3
n_r = 0
n_delta_functions = 2
{frozenset({(1, 2), (0, 3)}), frozenset({(0, 1), (2, 3)}), frozenset({(0, 2), (1, 3)})}
EinsteinTerm()
7
pref = -15
n_r = 2
n_delta_functions = 1
{frozenset({0,

# Numpy einsums

In [13]:
def get_np_einsum_first_arg(indices, n_indices):
    res = '"'

    alph = [chr(97 + i) for i in range(24)]

    items = []
    # first we append all the delta indices
    for i in indices:
        if not isinstance(i, int):
            print(i[0])
            items.append(f"{alph[i[0]]}{alph[i[1]]}")

    # then we append all the r indices
    for i in indices:
        if isinstance(i, int):
            items.append(f"{alph[i]}")

    res += insert_separator(items, ",")

    # lastly, the O index
    res += f" -> "
    for i in range(n_indices):
        res += f"{alph[i]}"

    res += '"'

    return res


def get_np_einsum_second_arg(einstein_term: EinsteinTerm):
    items = []

    for i in range(einstein_term.n_delta):
        items.append("delta")

    for i in range(einstein_term.n_r):
        items.append("r")

    return insert_separator(items, ", ")


def to_numpy(einstein_terms: list[EinsteinTerm]):
    result_string = ""

    n_summands = 0
    for t in einstein_terms:
        for p in t.permutations:
            np_einsum_first_arg = get_np_einsum_first_arg(p, 2 * t.n_delta + t.n_r)
            np_einsum_second_arg = get_np_einsum_second_arg(t)

            result_string += f"s{n_summands} = {t.prefactor:.1f}/d**{t.order} * np.einsum({np_einsum_first_arg}, {np_einsum_second_arg} )\n"
            n_summands += 1

    result_string += "return "

    items = [f"s{i}" for i in range(n_summands)]
    result_string += insert_separator(items, " + ")

    return result_string


output_path = base_output_path / "numpy"
output_path.mkdir(exist_ok=True, parents=True)
for n in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    with open(output_path / f"t{n}.py", "w") as f:
        f.write(to_numpy(T_Tensor(n)))

lowest_order = 1
highest_order = 1
r_scaling_exponent = 1
1
pref = 0
n_r = 0
n_delta_functions = 0
{frozenset()}
EinsteinTerm()
lowest_order = 3
highest_order = 3
r_scaling_exponent = 2
3
pref = -1
n_r = 1
n_delta_functions = 0
{frozenset({0})}
EinsteinTerm()
lowest_order = 3
highest_order = 5
r_scaling_exponent = 3
3
pref = -1
n_r = 0
n_delta_functions = 1
{frozenset({(0, 1)})}
EinsteinTerm()
5
pref = 3
n_r = 2
n_delta_functions = 0
{frozenset({0, 1})}
EinsteinTerm()
0
lowest_order = 5
highest_order = 7
r_scaling_exponent = 4
5
pref = 3
n_r = 1
n_delta_functions = 1
{frozenset({0, (1, 2)}), frozenset({(0, 1), 2}), frozenset({1, (0, 2)})}
EinsteinTerm()
7
pref = -15
n_r = 3
n_delta_functions = 0
{frozenset({0, 1, 2})}
EinsteinTerm()
1
0
0
lowest_order = 5
highest_order = 9
r_scaling_exponent = 5
5
pref = 3
n_r = 0
n_delta_functions = 2
{frozenset({(1, 2), (0, 3)}), frozenset({(0, 1), (2, 3)}), frozenset({(0, 2), (1, 3)})}
EinsteinTerm()
7
pref = -15
n_r = 2
n_delta_functions = 1
{froze

# C++ codegen

## Utils

In [14]:
def preamble(rank):
    res = f"""
/**
* @brief Rank {rank} Coulomb tensor.
*
* @tparam SW_Func_T
* @param r position difference vector
* @param sw_func switching function with signature sw_func(double, int) -> double
* @return Tensor<double>
*/
template <typename SW_Func_T>
    """

    items = rank * ["3"]
    threes = insert_separator(items, ",")

    res += f"inline Tensor<double, {threes}> T{rank}(const Tensor<double, 3>& r, const SW_Func_T& sw_func)"
    res += "\n{\n"

    res += "const double R1   = norm(r);\n"
    res += "const double R2   = R1*R1;\n"
    max_order = 2 * rank + 1

    if rank % 2 == 0:
        min_order = rank + 1
    else:
        min_order = rank + 2

    for o in range(3, max_order + 2, 2):
        res += f"const double R{o} = R{o-2} * R2;\n"

    for o in range(min_order, max_order + 2, 2):
        res += f"const double SW{o} = sw_func(R1, {o});\n"

    res += "using Special::delta;\n"

    return res

In [6]:
def get_einsum_template_arg(indices, n_indices):
    items = []

    # first we append all the delta indices
    for i in indices:
        if not isinstance(i, int):
            items.append(f"Index<{i[0]},{i[1]}>")

    # then we append all the r indices
    for i in indices:
        if isinstance(i, int):
            items.append(f"Index<{i}>")

    # lastly, the O index

    oindex = f"OIndex<"
    for i in range(n_indices):
        oindex += f"{i}"
        if i != n_indices - 1:
            oindex += ","
    oindex += ">"

    items.append(oindex)

    return insert_separator(items, ", ")

def get_einsum_args(einstein_term: EinsteinTerm):
    items = []
    for i in range(einstein_term.n_delta):
        items.append("delta")

    for i in range(einstein_term.n_r):
        items.append("r")

    return insert_separator(items, ", ")


def to_fastor(einstein_terms: list[EinsteinTerm]):
    result_string = ""

    n_summands = 0
    for t in einstein_terms:
        for p in t.permutations:
            einsum_template_arg = get_einsum_template_arg(p, 2 * t.n_delta + t.n_r)
            einsum_arg = get_einsum_args(t)

            result_string += f"const auto s{n_summands} = {t.prefactor:.1f}/R{t.order} * einsum<{einsum_template_arg}>( {einsum_arg} );\n"
            n_summands += 1

    items = [f"s{i}" for i in range(n_summands)]

    result_string += "return "
    result_string += insert_separator(items, " + ")
    result_string += ";"

    return result_string

output_path = base_output_path / "fastor"
output_path.mkdir(exist_ok=True, parents=True)
for n in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
    with open(f"t{n}.cpp", "w") as f:
        f.write(to_fastor(T_Tensor(n)))

lowest_order = 3
highest_order = 3
r_scaling_exponent = 2
3
pref = -1
n_r = 1
n_delta_functions = 0
{frozenset({0})}
EinsteinTerm()
lowest_order = 3
highest_order = 5
r_scaling_exponent = 3
3
pref = -1
n_r = 0
n_delta_functions = 1
{frozenset({(0, 1)})}
EinsteinTerm()
5
pref = 3
n_r = 2
n_delta_functions = 0
{frozenset({0, 1})}
EinsteinTerm()
lowest_order = 5
highest_order = 7
r_scaling_exponent = 4
5
pref = 3
n_r = 1
n_delta_functions = 1
{frozenset({0, (1, 2)}), frozenset({(0, 1), 2}), frozenset({1, (0, 2)})}
EinsteinTerm()
7
pref = -15
n_r = 3
n_delta_functions = 0
{frozenset({0, 1, 2})}
EinsteinTerm()
lowest_order = 5
highest_order = 9
r_scaling_exponent = 5
5
pref = 3
n_r = 0
n_delta_functions = 2
{frozenset({(1, 2), (0, 3)}), frozenset({(0, 1), (2, 3)}), frozenset({(0, 2), (1, 3)})}
EinsteinTerm()
7
pref = -15
n_r = 2
n_delta_functions = 1
{frozenset({0, 2, (1, 3)}), frozenset({1, 2, (0, 3)}), frozenset({(0, 1), 2, 3}), frozenset({(2, 3), 0, 1}), frozenset({0, 3, (1, 2)}), frozen

lowest_order = 11
highest_order = 19
r_scaling_exponent = 10
11
pref = -945
n_r = 1
n_delta_functions = 4
{frozenset({0, (4, 5), (2, 7), (3, 6), (1, 8)}), frozenset({(3, 8), (1, 2), (4, 5), 7, (0, 6)}), frozenset({2, (3, 8), (1, 5), (4, 7), (0, 6)}), frozenset({(0, 2), (3, 4), 7, (1, 5), (6, 8)}), frozenset({4, (3, 6), (2, 7), (1, 5), (0, 8)}), frozenset({4, (2, 7), (1, 5), (6, 8), (0, 3)}), frozenset({(2, 3), (6, 7), (0, 4), (1, 5), 8}), frozenset({(0, 1), (3, 8), (4, 5), (2, 6), 7}), frozenset({(0, 7), 5, (4, 6), (1, 3), (2, 8)}), frozenset({0, (2, 7), (5, 8), (4, 6), (1, 3)}), frozenset({(0, 2), (1, 7), 4, (3, 6), (5, 8)}), frozenset({2, (5, 6), (0, 3), (1, 8), (4, 7)}), frozenset({(4, 5), 7, (1, 6), (0, 3), (2, 8)}), frozenset({(6, 7), 4, (1, 5), (0, 3), (2, 8)}), frozenset({(2, 4), (1, 7), 5, (6, 8), (0, 3)}), frozenset({0, (2, 3), (5, 6), (1, 8), (4, 7)}), frozenset({1, (3, 8), (2, 7), (0, 5), (4, 6)}), frozenset({0, (3, 6), (2, 7), (1, 5), (4, 8)}), frozenset({(0, 2), (1, 7), 8,

## Using a callback function

In [15]:
def get_callback_summand(indices, n_indices):
    res = ""

    items = []

    # first we append all the delta indices
    for i in indices:
        if not isinstance(i, int):
            items.append(f"delta(indices[{i[0]}],indices[{i[1]}])")

    # then we append all the r indices
    for i in indices:
        if isinstance(i, int):
            items.append(f"r(indices[{i}])")

    return insert_separator(items, " * ")


def to_cpp_callback(einstein_terms: list[EinsteinTerm], rank):
    result_string = ""
    result_string += preamble(rank)

    result_string += r"// clang-format off" + "\n"
    result_string += (
        f"    auto callback = [&](const std::array<size_t, {rank}> & indices)"
    )
    result_string += "\n    {\n"

    n_summands = 0
    for t in einstein_terms:
        for p in t.permutations:
            einsum_template_arg = get_callback_summand(p, 2 * t.n_delta + t.n_r)

            result_string += f"        const auto s{n_summands} = {t.prefactor:.1f}/R{t.order} * SW{t.order} * {einsum_template_arg};\n"
            n_summands += 1

    items = [f"s{i}" for i in range(n_summands)]

    result_string += "        return "
    result_string += insert_separator(items, " + ")
    result_string += ";"
    result_string += "\n    };\n"
    result_string += r"// clang-format on" + "\n"

    items = rank * ["3"]
    threes = insert_separator(items, ",")
    result_string += f"\nreturn tensor_from_callback<decltype(callback), double, {threes}>(callback);"

    result_string += "\n}"
    return result_string


output_path = base_output_path / "fastor_callback"
output_path.mkdir(exist_ok=True, parents=True)

for n in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
    with open(output_path / f"t{n}_callback.cpp", "w") as f:
        f.write(to_cpp_callback(T_Tensor(n), n))

with open(output_path / f"callbacks.cpp", "w") as f:
    for n in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
        f.write(to_cpp_callback(T_Tensor(n), n))
        f.write("\n\n")

lowest_order = 3
highest_order = 3
r_scaling_exponent = 2
3
pref = -1
n_r = 1
n_delta_functions = 0
{frozenset({0})}
EinsteinTerm()
lowest_order = 3
highest_order = 5
r_scaling_exponent = 3
3
pref = -1
n_r = 0
n_delta_functions = 1
{frozenset({(0, 1)})}
EinsteinTerm()
5
pref = 3
n_r = 2
n_delta_functions = 0
{frozenset({0, 1})}
EinsteinTerm()
lowest_order = 5
highest_order = 7
r_scaling_exponent = 4
5
pref = 3
n_r = 1
n_delta_functions = 1
{frozenset({0, (1, 2)}), frozenset({(0, 1), 2}), frozenset({1, (0, 2)})}
EinsteinTerm()
7
pref = -15
n_r = 3
n_delta_functions = 0
{frozenset({0, 1, 2})}
EinsteinTerm()
lowest_order = 5
highest_order = 9
r_scaling_exponent = 5
5
pref = 3
n_r = 0
n_delta_functions = 2
{frozenset({(1, 2), (0, 3)}), frozenset({(0, 1), (2, 3)}), frozenset({(0, 2), (1, 3)})}
EinsteinTerm()
7
pref = -15
n_r = 2
n_delta_functions = 1
{frozenset({0, 2, (1, 3)}), frozenset({1, 2, (0, 3)}), frozenset({(0, 1), 2, 3}), frozenset({(2, 3), 0, 1}), frozenset({0, 3, (1, 2)}), frozen